## 5.5 TensorFlow最佳实践样例程序
在5.2.1节中已经给出了一个完整的TensorFlow程序来解决MNIST问题。然而如5.3节和5.4节所述，这个程序的可扩展性并不好，而且也没有持久化训练好的模型。结合前两节介绍的变量管理机制和持久化机制，本节给出了一个TensorFlow训练神经网络模型的最佳实践。

在本样例程序中，**将训练和测试分成两个独立的程序，这可以使得每个组件更加灵活**，比如训练神经网络的程序可以持续输出训练好的模型，而测试程序可以每隔一段时间检验最新模型的正确率，如果模型效果更好，则将这个模型提供给产品使用。另外因为在训练和测试过程中都会用到前向传播，本样例中还**将前向传播的过程抽象成一个单独的库函数，这样使用更加方便，也可以保证两个过程中使用到的前向传播是一致的。**

本节将提供重构之后的程序来解决MNIST问题，重构之后的代码将本拆分为3个程序：
- 第一个是mnist_inferrnce.py，它定义了前向传播的过程和神经网络中的参数；
- 第二个是mnist_train.py，它定义了神经网络的训练过程；
- 第三个是mnist_eval.py，它定义了测试过程。

前两个文件见本文件同目录，第三个文件直接给出在本文件中，如下：

*（需要说明的是，由于本人电脑是GTX1060 6G版，同时运行train和eval会报错：`InternalError: Blas SGEMM launch failed`。参考[here](https://blog.csdn.net/Vinsuan1993/article/details/81142855)，于是需要使用`gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.4) `在初始化Session的时候为其分配固定数量的显存，这里分别分配0.5/0.4）*

In [1]:
import time
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# 加载mnist_inference.py和mnist_train.py中定义的常量和函数
import mnist_inference
import mnist_train


# 每10秒加载一次最新模型，并在测试数据上测试最新模型的正确率
EVAL_INTERVAL_SECS = 10

def evaluate(mnist):
    with tf.Graph().as_default() as g:
        # 1. 定义神经网络输入输出
        x = tf.placeholder(tf.float32, [None, mnist_inference.INPUT_NODE], name='x-input')
        y_ = tf.placeholder(tf.float32, [None, mnist_inference.OUTPUT_NODE], name='y-input')
        validate_feed = {x: mnist.validation.images, y_: mnist.validation.labels}

        # 2. 计算前向传播结果
        y = mnist_inference.inference(x, None)
        
        # 3. 计算前向传播的结果的正确率，如果需要对未知的样例进行分了，那么使用tf.argmax(y, 1)就可以得到预测类别
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

        # 通过变量重命名的方式来加载模型，这样在前向传播的过程中就不需要调用求滑动平均的
        # 函数来获取平均值了。这样就可以完全共用mnist_inference.py 中定义的前向传播过程。
        variable_averages = tf.train.ExponentialMovingAverage(mnist_train.MOVING_AVERAGE_DECAY)
        variables_to_restore = variable_averages.variables_to_restore()
        saver = tf.train.Saver(variables_to_restore)
        
        # 每隔EVAL_INTERVAL_SECS秒调用一次计算正确率的过程，并检测训练过程中的正确率变化
        while True:
            gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.4)    # 给当前session分配固定数量显存
            with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
                # tf.train.get_checkpoint_state函数会通过checkpoint文件自动找到目录中最新的文件名
                ckpt = tf.train.get_checkpoint_state(mnist_train.MODEL_SAVE_PATH)
                if ckpt and ckpt.model_checkpoint_path:
                    # 加载模型
                    saver.restore(sess, ckpt.model_checkpoint_path)
                    # 通过文件名得到模型保存时迭代的轮数
                    global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
                    accuracy_score = sess.run(accuracy, feed_dict=validate_feed)
                    print("After %s training step(s), validation accuracy = %g" % (global_step, accuracy_score))
                else:
                    print('No checkpoint file found')
                    return
                
            time.sleep(EVAL_INTERVAL_SECS)
            

def main(argv=None):
    mnist = input_data.read_data_sets("../../../datasets/MNIST_data", one_hot=True)
    evaluate(mnist)

if __name__ == '__main__':
    main()

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../../datasets/MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../../datasets/MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../../../datasets/MNIST_data\t10k-images-idx3-ubyte.gz
Extracting ../../../datasets/MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
INFO:tensorflow:Restoring parameters from MNIST_model/mnist_model-13001
After 13001 training step(s), validation accuracy = 0.984
INFO:tensorflow:Restoring parameters from MNIST_model/mnist_model-18001
After 18001 training step(s

KeyboardInterrupt: 